## Pipeline to search best model

In [1]:
import mlflow
from mlflow.tracking import MlflowClient
import mlflow.sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from time import time
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, learning_curve, validation_curve
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sample_df = pd.read_csv('../data/sample_df.csv')

In [5]:
## Preprocessing sample_df: scaling
### METTRE DANS LA PIPELINE - A FAIRE
#remplacer les valeurs infinies par des NaN
sample_df.replace([np.inf, -np.inf], np.nan, inplace=True)

#sépare la cible du reste des données
X = sample_df.drop(columns=["TARGET"])
y = sample_df['TARGET']
print(f"shape de X : {X.shape}")
print(f"shape de y : {y.shape}")

# Remplacement des valeurs manquantes par le mode (la valeur la plus fréquente)
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = imputer.fit_transform(X)

# Standardisation des caractéristiques
# Cela pourrait être utile pour certains modèles qui sont sensibles aux valeurs abbérantes
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_imputed)

# on construit les échantillons d'apprentissage et de validation
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2,random_state=42, stratify=y)#stratify to preserve the proportion of target as in the original dataset


X_train_df = pd.DataFrame(X_train, columns=X.columns)
print(X_train_df.isnull().sum())

shape de X : (4315, 797)
shape de y : (4315,)
index                               0
SK_ID_CURR                          0
CODE_GENDER                         0
FLAG_OWN_CAR                        0
FLAG_OWN_REALTY                     0
                                   ..
CC_NAME_CONTRACT_STATUS_nan_MAX     0
CC_NAME_CONTRACT_STATUS_nan_MEAN    0
CC_NAME_CONTRACT_STATUS_nan_SUM     0
CC_NAME_CONTRACT_STATUS_nan_VAR     0
CC_COUNT                            0
Length: 797, dtype: int64


## Pipeline 1

In [62]:
# Créez vos pipelines
pipe_lr = Pipeline([('clf', LogisticRegression())])
pipe_dt = Pipeline([('clf', DecisionTreeClassifier())])
pipe_rf = Pipeline([('clf', RandomForestClassifier())])
pipe_svm = Pipeline([('clf', SVC())])

# Set grid search params
param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range_fl = [1.0, 0.5, 0.1]

grid_params_lr = [{'clf__penalty': ['l1', 'l2'],
                   'clf__C': param_range_fl,
                   'clf__solver': ['liblinear']}] 

grid_params_dt = [{'clf__criterion': ['gini', 'entropy'],
                   'clf__min_samples_leaf': param_range,
                   'clf__max_depth': param_range,
                   'clf__min_samples_split': param_range[1:]}]

grid_params_rf = [{'clf__criterion': ['gini', 'entropy'],
                   'clf__min_samples_leaf': param_range,
                   'clf__max_depth': param_range,
                   'clf__min_samples_split': param_range[1:]}]

grid_params_svm = [{'clf__kernel': ['linear', 'rbf'], 
                    'clf__C': param_range}]

# Construct grid searches
jobs = -1

gs_lr = GridSearchCV(estimator=pipe_lr,
                     param_grid=grid_params_lr,
                     scoring='roc_auc',
                     cv=10) 

gs_dt = GridSearchCV(estimator=pipe_dt,
                     param_grid=grid_params_dt,
                     scoring='roc_auc',
                     cv=10,
                     n_jobs=jobs)

gs_rf = GridSearchCV(estimator=pipe_rf,
                     param_grid=grid_params_rf,
                     scoring='roc_auc',
                     cv=10, 
                     n_jobs=jobs)

gs_svm = GridSearchCV(estimator=pipe_svm,
                      param_grid=grid_params_svm,
                      scoring='roc_auc',
                      cv=10,
                      n_jobs=jobs)

# List of pipelines for ease of iteration
grids = [gs_lr, gs_dt, gs_rf, gs_svm]

# Dictionary of pipelines and classifier types for ease of reference
grid_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'Random Forest', 3: 'Support Vector Machine'}

# Fit the grid search objects
print('Performing model optimizations...')
best_auc = 0.0
best_clf = 0
best_gs = ''

for idx, gs in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx])	
    gs.fit(X_train, y_train)
    print('Best params: %s' % gs.best_params_)
    print('Best training accuracy: %.3f' % gs.best_score_)
    # Predict on test data with best params
    y_pred_proba = gs.predict_proba(X_test)[:, 1]
    # Test data accuracy of model with best params
    print('Test set accuracy score for best params: %.3f ' % roc_auc_score(y_test, y_pred_proba))
    # Track best (highest test accuracy) model
    if roc_auc_score(y_test, y_pred_proba) > best_auc:
        best_auc = roc_auc_score(y_test, y_pred_proba)
        best_gs = gs
        best_clf = idx
print('\nClassifier with best test set AUC: %s' % grid_dict[best_clf])

Performing model optimizations...

Estimator: Logistic Regression


c:\Users\dalila.derdar.ONCEFORALL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\dalila.derdar.ONCEFORALL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\dalila.derdar.ONCEFORALL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\dalila.derdar.ONCEFORALL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\dalila.derdar.ONCEFORALL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:124

Best params: {'clf__C': 0.1, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
Best training accuracy: 0.673
Test set accuracy score for best params: 0.701 

Estimator: Decision Tree
Best params: {'clf__criterion': 'entropy', 'clf__max_depth': 5, 'clf__min_samples_leaf': 4, 'clf__min_samples_split': 5}
Best training accuracy: 0.628
Test set accuracy score for best params: 0.521 

Estimator: Random Forest
Best params: {'clf__criterion': 'entropy', 'clf__max_depth': 7, 'clf__min_samples_leaf': 9, 'clf__min_samples_split': 9}
Best training accuracy: 0.721
Test set accuracy score for best params: 0.689 

Estimator: Support Vector Machine


## Pipeline 2

In [ ]:
# Initialize classifiers
logreg = LogisticRegression(random_state=42)
tree = DecisionTreeClassifier(random_state=42)
forest = RandomForestClassifier(random_state=42)
svm = SVC(random_state=42, probability=True)

# Define the hyperparameters
hyperparameters = {
    logreg: {
        'C': np.logspace(-4, 4, 20),
        'solver': ['liblinear']
    },
    tree: {
        'criterion': ['gini', 'entropy'],
        'max_depth': range(1, 10),
        'min_samples_split': range(2, 10),
        'min_samples_leaf': range(1, 10)
    },
    forest: {
        'n_estimators': range(10, 101, 10),
        'criterion': ['gini', 'entropy'],
        'max_depth': range(1, 10),
        'min_samples_split': range(2, 10),
        'min_samples_leaf': range(1, 10)
    },
    svm: {
        'C': [0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001],
        'kernel': ['rbf', 'poly', 'sigmoid']
    }
}

# Initialize the DataFrame for storing the results
results = []

# Loop through the classifiers
for classifier, params in hyperparameters.items():
    start = time()
    classifier_name = classifier.__class__.__name__
    
    gs = RandomizedSearchCV(classifier, params, scoring='roc_auc', cv=5, n_iter=10, n_jobs=-1, random_state=42)
    gs.fit(X_train, y_train)
    
    # Predict the target for the testing data
    y_pred = gs.predict(X_test)
    
    # Calculate the metrics
    auc = roc_auc_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Store the results in a DataFrame
    results.append({
        'Model': classifier_name,
        'Best Parameters': gs.best_params_,
        'Execution Time (s)': time() - start,
        'Training AUC': gs.best_score_,
        'Testing AUC': auc,
        'Testing Accuracy': acc,
        'Testing F1 Score': f1
    })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
results_df


c:\Users\dalila.derdar.ONCEFORALL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Pipeline 3

### Pipeline 3.1 : tous les modèles avec paramètres par défaut 

In [19]:
# Initialize the classifiers
clf1 = LogisticRegression(random_state=42)
clf2 = DecisionTreeClassifier(random_state=42)
clf3 = RandomForestClassifier(random_state=42)
clf4 = SVC(random_state=42, probability=True)
clf5 = KNeighborsClassifier()

# Initialize the hyperparameters for each dictionary
param1 = {'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'clf__penalty': ['l2'], 'clf': [clf1]}
param2 = {'clf__max_depth': [None, 5, 10, 15, 20], 'clf__min_samples_split': [2, 5, 10], 'clf': [clf2]}
param3 = {'clf__n_estimators': [10, 50, 100, 200], 'clf__max_depth': [None, 5, 10, 15, 20], 'clf': [clf3]}
param4 = {'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'clf__gamma': ['scale', 'auto'], 'clf': [clf4]}
param5 = {'clf__n_neighbors': [3, 5, 7, 9, 11], 'clf': [clf5]}

## rajouter ici les preprocessing

# Create a pipeline
pipeline = Pipeline([('clf', clf1)])
params = [param1, param2, param3, param4, param5]

#Demander de calculer d'autres métrics sur le valide set
scorings = ['roc_auc', 'precision', 'recall', 'f1']

# Use RandomizedSearchCV to find the best parameters
gs = RandomizedSearchCV(pipeline, params, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1, scoring=scorings,refit='roc_auc', return_train_score=True)

start = time.time()

# Fit the model
gs.fit(X_train, y_train)

print("Execution time: " + str((time.time() - start)) + ' ms')
print()
print("Best parameters: ", gs.best_params_)
print()
print("Best training AUC: ", gs.best_score_)
print()
# Predict the labels of the test set
y_pred = gs.predict(X_test)

# Calculate the probabilities for the test set
y_pred_proba = gs.predict_proba(X_test)[:, 1]

# Compute the AUC score, precision, recall, and F1 score
auc_test = roc_auc_score(y_test, y_pred_proba)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Store the results in a DataFrame
results = pd.DataFrame({
    'Model': [gs.best_params_['clf'].__class__.__name__],
    'Best Parameters': [gs.best_params_],
    'Training AUC': [gs.best_score_],
    'Test AUC': [auc_test],
    'Precision': [precision],
    'Recall': [recall],
    'F1 Score': [f1]
})

print(results)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Execution time: 352.74126839637756 ms

Best parameters:  {'clf__n_estimators': 100, 'clf__max_depth': 20, 'clf': RandomForestClassifier(max_depth=20, random_state=42)}

Best training AUC:  0.6817312086060937

                    Model                                    Best Parameters  \
0  RandomForestClassifier  {'clf__n_estimators': 100, 'clf__max_depth': 2...   

   Training AUC  Test AUC  Precision  Recall  F1 Score  
0      0.681731  0.642857        0.0     0.0       0.0  


c:\Users\dalila.derdar.ONCEFORALL\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
cm_1 = confusion_matrix(y_test, y_pred)
cm_1

array([[793,   0],
       [ 70,   0]], dtype=int64)

In [20]:
results 

,Model,Best Parameters,Training AUC,Test AUC,Precision,Recall,F1 Score
0,RandomForestClassifier,"{'clf__n_estimators': 100, 'clf__max_depth': 2...",0.681731,0.642857,0.0,0.0,0.0


### Pipeline 3.2 : recherche des meilleurs hyerparamètres sur le meilleur modèle randomforest

#### Pipeline 3-2-1 recherche des meilleurs hyerparamètres sur le meilleur modèle randomforest - score roc_auc

In [8]:
#Réentrainer avec plus d'hyperparamètres de Random Forest

# Define the hyperparameters
param_dist = {
    'n_estimators': randint(10, 200),
    'max_depth': randint(1, 20),
    'class_weight': ['balanced', None],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}

# Initialize the classifier
clf = RandomForestClassifier(random_state=42)

scorings = ['roc_auc', 'precision', 'recall', 'f1']
# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, cv=5, n_jobs=-1, scoring= scorings, refit= 'roc_auc')

# Fit the model
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters: ", random_search.best_params_)

# Predict the labels of the test set
y_pred = random_search.predict(X_test)

# Evaluate on the test set
y_pred_proba = random_search.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Store the results in a DataFrame
results = pd.DataFrame({
    'Model': [random_search.best_estimator_.__class__.__name__],
    'Best Parameters': [random_search.best_params_],
    'Training AUC': [random_search.best_score_],
    'Test AUC': [auc],
    'Precision': [precision],
    'Recall': [recall],
    'F1 Score': [f1]
})

print(results)


Best parameters:  {'class_weight': 'balanced', 'max_depth': 14, 'max_features': None, 'min_samples_leaf': 8, 'min_samples_split': 5, 'n_estimators': 155}
                    Model                                    Best Parameters  \
0  RandomForestClassifier  {'class_weight': 'balanced', 'max_depth': 14, ...   

   Training AUC  Test AUC  Precision    Recall  F1 Score  
0      0.710788  0.685246   0.333333  0.071429  0.117647  


In [9]:
#afficher la matrice de confusion pour le meilleur modèle
#matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[783  10]
 [ 65   5]]


#### Pipeline 3-2-2 recherche des meilleurs hyerparamètres sur le meilleur modèle randomforest - score recall

In [6]:
#Réentrainer avec plus d'hyperparamètres de Random Forest

# Define the hyperparameters
param_dist = {
    'n_estimators': randint(10, 200),
    'max_depth': randint(1, 20),
    'class_weight': ['balanced', None],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}

# Initialize the classifier
clf = RandomForestClassifier(random_state=42)

scorings = ['roc_auc', 'precision', 'recall', 'f1']
# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, cv=5, n_jobs=-1, scoring= scorings, refit= 'recall')

# Fit the model
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters: ", random_search.best_params_)

# Predict the labels of the test set
y_pred = random_search.predict(X_test)

# Evaluate on the test set
y_pred_proba = random_search.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Store the results in a DataFrame
results = pd.DataFrame({
    'Model': [random_search.best_estimator_.__class__.__name__],
    'Best Parameters': [random_search.best_params_],
    'Training AUC': [random_search.best_score_],
    'Test AUC': [auc],
    'Precision': [precision],
    'Recall': [recall],
    'F1 Score': [f1]
})

print(results)


Best parameters:  {'class_weight': 'balanced', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 9, 'n_estimators': 195}
                    Model                                    Best Parameters  \
0  RandomForestClassifier  {'class_weight': 'balanced', 'max_depth': 2, '...   

   Training AUC  Test AUC  Precision    Recall  F1 Score  
0      0.572338  0.696379   0.131429  0.657143  0.219048  


In [7]:
#afficher la matrice de confusion pour le meilleur modèle
#matrice de confusion
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[489 304]
 [ 24  46]]


### Pipeline 3-3 modification des hyperparamètres du meilleur modèle randomforest

In [16]:
#Réentrainer avec changement de seuil de décision à 0.3

# Define the hyperparameters
param_dist = {
    'n_estimators': randint(10, 200),
    'max_depth': randint(1, 20),
    'class_weight': ['balanced', None],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}

# Initialize the classifier
clf = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, cv=5, n_jobs=-1)

# Fit the model
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters: ", random_search.best_params_)

# Predict the labels of the test set with a different threshold
y_pred_proba = random_search.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.3).astype(int)

# Evaluate on the test set
auc = roc_auc_score(y_test, y_pred_proba)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Store the results in a DataFrame
results = pd.DataFrame({
    'Model': [random_search.best_estimator_.__class__.__name__],
    'Best Parameters': [random_search.best_params_],
    'Training AUC': [random_search.best_score_],##ce n'est pas le score train score mais le score de validation. Plutôt faire y_train_pred = random_search.predict(X_train) 
    'Test AUC': [auc],
    'Precision': [precision],
    'Recall': [recall],
    'F1 Score': [f1]
})

print(results)


Best parameters:  {'class_weight': None, 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 8, 'min_samples_split': 9, 'n_estimators': 148}
                    Model                                    Best Parameters  \
0  RandomForestClassifier  {'class_weight': None, 'max_depth': 10, 'max_f...   

   Training AUC  Test AUC  Precision    Recall  F1 Score  
0      0.919758   0.71207     0.3125  0.071429  0.116279  


Le score d'AUC pour le jeu d'entraînement est de 0.92, ce qui indique que le modèle est capable de distinguer efficacement les remboursements des non-remboursements sur le jeu de données d'entraînement. Cependant, le score d'AUC pour le jeu de test est de 0.71, ce qui est significativement plus bas. Cela peut indiquer que le modèle surapprend sur le jeu de données d'entraînement (overfitting) et ne généralise pas bien aux nouvelles données.

La précision est de 0.31, ce qui signifie que seulement 31% des prêts que le modèle prédit comme étant en défaut (1 = ne peut pas rembourser) sont effectivement en défaut. En d'autres termes, beaucoup de prêts qui auraient pu être remboursés sont refusés par le modèle.

Le recall est de 0.071, ce qui signifie que le modèle est seulement capable de détecter environ 7.1% des défauts de remboursement. En d'autres termes, le modèle laisse passer environ 93% des défauts de remboursement, ce qui est très problématique car cela signifie que beaucoup de prêts qui ne seront pas remboursés sont accordés.

Le score F1 est une moyenne harmonique de la précision et du rappel, et il est généralement plus utile que la précision lorsque vous avez une répartition inégale des classes (c'est-à-dire beaucoup plus de remboursements que de défauts de remboursement). Un score F1 de 0.116 est assez faible, ce qui indique que le modèle ne parvient pas à trouver un bon équilibre entre précision et rappel.

ces résultats suggèrent que votre modèle a du mal à identifier les défauts de remboursement, et il est probablement trop prudent en refusant beaucoup de prêts qui seraient effectivement remboursés. Cela pourrait être dû à une sous-représentation des défauts de remboursement dans les données, ou à un seuil de décision qui est trop élevé.

Essayer le rééquilibrage des classes

### Pipeline 3-4 Randomforest et test hyperparamètres - intégration preprocessing + ML FLow

In [14]:
# Lire les données
sample_df = pd.read_csv('../data/sample_df.csv')

# Remplacer les valeurs infinies par des NaN
sample_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Séparer la cible du reste des données
X = sample_df.drop(columns=["TARGET"])
y = sample_df['TARGET']
print(f"shape de X : {X.shape}")
print(f"shape de y : {y.shape}")

# Construire le pipeline de prétraitement
preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', RobustScaler())
])

# Application du prétraitement et split du dataset
X = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

###Setup MLFLOW
mlflow.set_experiment('Scoring_models')
mlflow.sklearn.autolog()

# Définir les hyperparamètres
param_dist = {
    'n_estimators': randint(10, 200),
    'max_depth': randint(1, 20),
    'class_weight': ['balanced', None],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}

# Initialiser le classifieur
clf = RandomForestClassifier(random_state=42)

scorings = ['roc_auc', 'precision', 'recall', 'f1']

# Initialiser RandomizedSearchCV
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, cv=5, n_jobs=-1, scoring=scorings, refit='recall')

# Start an MLflow run
with mlflow.start_run(run_name='test_1'):

    # Entraîner le modèle
    random_search.fit(X_train, y_train)

    # Get the best model
    best_model = random_search.best_estimator_

    # Calculate train score
    train_score = recall_score(y_train, best_model.predict(X_train))

    # Calculate validation score (best_score_ is already based on validation set because we're using cross-validation in RandomizedSearchCV)
    validation_score = random_search.best_score_

    # Prédire les labels de l'ensemble de test
    y_pred = random_search.predict(X_test)

    # Evaluate on the test set
    test_score = recall_score(y_test, y_pred)

    # Evaluer sur l'ensemble de test
    y_pred_proba = random_search.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log the parameters
    mlflow.log_params(random_search.best_params_)


    # Log the metrics
    mlflow.log_metric("Training Recall", train_score)
    mlflow.log_metric("Validation Recall", validation_score)
    mlflow.log_metric("Test Recall", test_score)
    mlflow.log_metric("Test AUC", auc)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1 Score", f1)

    # Log the model
    mlflow.sklearn.log_model(best_model, "model")
    # Stocker les résultats dans un DataFrame
    results = pd.DataFrame({
        'Model': [best_model.__class__.__name__],
        'Best Parameters': [random_search.best_params_],
        'Training AUC': [random_search.best_score_],
        'Test AUC': [auc],
        'Precision': [precision],
        'Recall': [recall],
        'F1 Score': [f1], 
        'Training Recall': [train_score],
        'Validation Recall': [validation_score],
        'Test Recall': [test_score]
    })

print(results)


2023/07/10 16:51:41 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a6d4f388e5094b62938ca6fffa1838ec', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


shape de X : (4315, 797)
shape de y : (4315,)


2023/07/10 16:51:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\DalilaDerdar\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\data\pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/07/10 16:51:43 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were no

                    Model                                    Best Parameters  \
0  RandomForestClassifier  {'class_weight': 'balanced', 'max_depth': 2, '...   

   Training AUC  Test AUC  Precision    Recall  F1 Score  Training Recall  \
0      0.417208  0.657737   0.153846  0.485714  0.233677         0.640288   

   Validation Recall  Test Recall  
0           0.417208     0.485714  


### Pipeline 3-5 Randomforest, test hyperparamètres - pipeline preprocessing & classifier + ML FLow

In [7]:
# Read data
sample_df = pd.read_csv('../data/sample_df.csv')

# Replace infinite values with NaN
sample_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Separate the target from the rest of the data
X = sample_df.drop(columns=["TARGET"])
y = sample_df['TARGET']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### Setup MLFLOW
mlflow.set_experiment('Scoring_models')
mlflow.sklearn.autolog()

# Define preprocessing pipeline
preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', RobustScaler())
])

# Define classifier
clf = RandomForestClassifier(random_state=42)

# Combine preprocessing and classifier into a single pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', clf)
])

# Define hyperparameters for the Random Forest Classifier
param_dist = {
    'classifier__n_estimators': randint(100, 200),
    'classifier__max_depth': randint(1,5),
    'classifier__class_weight': ['balanced', 'balanced_subsample'],
    'classifier__min_samples_split': randint(20,40),
    'classifier__min_samples_leaf': randint(10,40),
    'classifier__max_features': ['sqrt', 'log2', None]
}

scorings = ['roc_auc', 'precision', 'recall', 'f1']

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, cv=5,n_jobs=-1, scoring=scorings, refit='recall', n_iter=20)

# Start an MLflow run
with mlflow.start_run(run_name='Modelisation_test6'):

    # Train the model
    random_search.fit(X_train, y_train)

    # Get the best model
    best_model = random_search.best_estimator_

    # Calculate train score
    train_score = recall_score(y_train, best_model.predict(X_train))

    # Calculate validation score
    validation_score = random_search.best_score_

    # Predict the labels of the test set
    y_pred = random_search.predict(X_test)

    # Evaluate on the test set
    test_score = recall_score(y_test, y_pred)

    # Evaluate on the test set
    y_pred_proba = random_search.predict_proba(X_test)[:, 1]
    threshold = 0.4
    y_pred = (y_pred_proba > threshold).astype(int)
    auc = roc_auc_score(y_test, y_pred_proba)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log the parameters
    mlflow.log_params(random_search.best_params_)

    # Log the metrics
    mlflow.log_metric("Training Recall", train_score)
    mlflow.log_metric("Validation Recall", validation_score)
    mlflow.log_metric("Test Recall", test_score)
    mlflow.log_metric("Test AUC", auc)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1 Score", f1)

    # Log the model
    mlflow.sklearn.log_model(best_model, "model")

    # Store the results in a DataFrame
    results = pd.DataFrame({
        'Model': [random_search.best_estimator_.named_steps['classifier'].__class__.__name__],
        'Best Parameters': [random_search.best_params_],
        'Training Recall': [train_score],
        'Validation Recall': [validation_score],
        'Test Recall': [test_score],
        'Test AUC': [auc],
        'Precision': [precision],
        'F1 Score': [f1] 
    })

print(results)


2023/07/14 19:04:05 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2023/07/14 19:04:06 WARNING mlflow.utils: Truncated the value of the key `param_distributions`. Truncated value: `{'classifier__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000019DC140E7D0>, 'classifier__max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000019DEA91A5C0>, 'classifier__class_weight': ['balanced', 'balanced_subsample'], 'classifier__min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000019DEA91B5E0>, 'classifier__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen objec...`
2023/07/14 19:04:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\DalilaDerdar\AppData\Local\Program

                    Model                                    Best Parameters  \
0  RandomForestClassifier  {'classifier__class_weight': 'balanced', 'clas...   

   Training Recall  Validation Recall  Test Recall  Test AUC  Precision  \
0         0.676259           0.586818     0.657143  0.670159   0.098592   

   F1 Score  
0  0.177715  


#### Voir l'ensemble des expériences dans MLFlow

In [12]:
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("file:///C:/Users/DalilaDerdar/OneDrive - Supplier Assessment Services Limited/Bureau/Scoring_Model/notebook/mlruns")

client = MlflowClient()

experiments = client.search_experiments() # retourne une liste d'objets Experiment
for experiment in experiments:
    print("Name: {}, Experiment_id: {}".format(experiment.name, experiment.experiment_id))


Name: mlflow-demo3, Experiment_id: 967850129774567930
Name: mlflow-demo2, Experiment_id: 378466946012600378
Name: mlflow-demo, Experiment_id: 986021913707031087
Name: Default, Experiment_id: 0
